#  Машинный перевод с использованием рекуррентных нейронных сетей и механизма внимания

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-using-attention-with-pytorch/
* http://ethen8181.github.io/machine-learning/deep_learning/seq2seq/2_torch_seq2seq_attention.html
* https://tomekkorbak.com/2020/06/26/implementing-attention-in-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример работы `torch.nn.utils.rnn.pad_sequence` для генерации батча.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch as th


class FakeDset(Dataset):
    def __init__(self):
        self.x = [
            th.LongTensor([2, 27, 705, 2327, 5744, 3]),
            th.LongTensor([2, 7, 29, 240, 5669, 2747, 1479, 3]),
            th.LongTensor([2, 27, 705, 2327, 3]),
            th.LongTensor([2, 7, 29, 240, 5669, 2747, 1479, 7, 29, 240, 5669, 3]),
        ]

    def __getitem__(self, idx):
        return (self.x[idx],)

    def __len__(self):
        return len(self.x)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class Collator:
  def __init__(self, pad_idx, batch_first=True):
    self.pad_idx = pad_idx
    self.batch_first = batch_first

  def __call__(self, batch):
    x = [x[0] for x in batch]
    x = pad_sequence(
        x,
        batch_first=self.batch_first,
        padding_value=self.pad_idx
    )
    return x

In [ ]:
dset = FakeDset()
loader = DataLoader(
    dset,
    batch_size=2,
    collate_fn=Collator(batch_first=True, pad_idx=0)
)

In [ ]:
next(iter(loader))

tensor([[   2,   27,  705, 2327, 5744,    3,    0,    0],
        [   2,    7,   29,  240, 5669, 2747, 1479,    3]])

2\. Рассмотрите основные шаги для реализации механизма аддитивного внимания с использованием RNN.

![attention](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2F23YEOESycSImB4FBnFS09C_R18ffuD9luGO4X4cpF4Unqon5-l6fuWwsvottjU_Aj8&usqp=CAU)

$$c_i = \sum_{j=1}^{T_x}\alpha_{ij}h_j$$

$$\alpha_{ij} = \frac{exp(e_{ij})}{\sum_{j=1}^{T_x}exp(e_{ik})}$$

$$ e_{ij} = a(s_{i-1},h_j) = v_a^Ttanh(W_a s_{i-1} + U_a h_j)$$

In [ ]:
import torch as th

In [ ]:
batch_size, input_seq_len = 16, 5
encoder_hidden_dim = 100

# X: batch_size x input_seq_len
# encoder(X)
encoder_outputs = th.rand(batch_size, input_seq_len, encoder_hidden_dim)
encoder_final_hidden = encoder_outputs[:, -1, :].unsqueeze(0)
# encoder_outputs: batch_size, input_seq_len x encoder_hidden_dim
# encoder_final_hidden: 1 x batch_size x encoder_hidden_dim

In [ ]:
encoder_outputs.shape, encoder_final_hidden.shape

(torch.Size([16, 5, 100]), torch.Size([1, 16, 100]))

In [ ]:
import torch.nn as nn

decoder_hidden_dim = 100

decoder_input = th.rand(batch_size, encoder_hidden_dim)
decoder_hidden = encoder_final_hidden[0]
rnn = nn.GRUCell(encoder_hidden_dim, decoder_hidden_dim)
rnn(decoder_input, decoder_hidden).shape

torch.Size([16, 100])

In [ ]:
decoder_hidden = encoder_final_hidden
decoder_hidden = (
    decoder_hidden
    .repeat(input_seq_len, 1, 1)
    .permute(1, 0, 2)
) # batch x input_seq_len x encoder_hidden
decoder_hidden.shape

torch.Size([16, 5, 100])

In [ ]:
encoder_with_hidden = th.cat(
    (decoder_hidden, encoder_outputs),
    dim=2
)
encoder_with_hidden.shape

torch.Size([16, 5, 200])

In [ ]:
fc1 = nn.Linear(200, decoder_hidden_dim)

attn_hidden = fc1(encoder_with_hidden).tanh()
# attn_hidden : batch x input_seq_len x decoder_hidden_dim

fc2 = nn.Linear(decoder_hidden_dim, 1)
energy = fc2(attn_hidden).squeeze(2)
# energy: batch x input_seq x 1
attn = energy.softmax(dim=1)
# attn : batch x input_seq

In [ ]:
# encoder_outputs: batch_size x input_seq_len x encoder_hidden_dim
attn = attn.unsqueeze(1)
# attn : batch_size x 1 x input_seq_len
weighted_encoder_outputs = th.bmm(attn, encoder_outputs).squeeze(1)
weighted_encoder_outputs.shape

torch.Size([16, 100])

In [ ]:
# возвращаемся сюда
decoder_input = th.rand(batch_size, encoder_hidden_dim)

rnn_input = th.cat(
    (decoder_input, weighted_encoder_outputs),
    dim=1
)
rnn_input.shape

torch.Size([16, 200])

In [ ]:
decoder_hidden = encoder_final_hidden[0]
rnn = nn.GRUCell(200, decoder_hidden_dim)
decoder_hidden = rnn(rnn_input, decoder_hidden)

In [ ]:
n_en_words = 1337
fc_out = nn.Linear(100, n_en_words)

logits = fc_out(decoder_hidden)
logits.shape

torch.Size([16, 1337])

In [ ]:
logits.argmax(dim=1).shape
# идем наверх

torch.Size([16])

## Задачи для самостоятельного решения

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install torchmetrics

In [ ]:
import json
import random
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import numpy as np
import re
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchmetrics as M
from sklearn.metrics import classification_report
from torchtext.data.metrics import bleu_score
from nltk.translate.bleu_score import corpus_bleu

<p class="task" id="1"></p>

### 1
Создайте наборы данных для решения задачи машинного перевода на основе файлов `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json`. При подготовке набора данных не приводите весь набор данных к одинаковой фиксированной длине. Реализуйте класс `Collator`, который приводит все примеры в батче к одной фиксированной длине, используя `torch.nn.utils.rnn.pad_sequence`. Создайте `DataLoader` с использованием `collate_fn`, получите батч и выведите на экран размер тензоров.

- [x] Проверено на семинаре

In [ ]:
with open('/content/drive/MyDrive/data/RuBQ_2.0_train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

data_train = pd.DataFrame(data)
data_train = data_train[['question_text', 'question_eng']]
data_train.head()

,question_text,question_eng
0,Что может вызвать цунами?,What can cause a tsunami?
1,Кто написал роман «Хижина дяди Тома»?,"Who wrote the novel ""uncle Tom's Cabin""?"
2,Кто автор пьесы «Ромео и Джульетта»?,"Who is the author of the play ""Romeo and Juliet""?"
3,Как называется столица Румынии?,What is the name of the capital of Romania?
4,На каком инструменте играл Джимми Хендрикс?,What instrument did Jimi Hendrix play?


In [ ]:
with open('/content/drive/MyDrive/data/RuBQ_2.0_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

data_test = pd.DataFrame(data)
data_test = data_test[['question_text', 'question_eng']]
data_test.head()

,question_text,question_eng
0,Какой стране принадлежит знаменитый остров Пасхи?,Which country does the famous Easter island be...
1,С какой музыкальной группой неразрывно связано...,Which music group is Mick Jagger's name inextr...
2,Где находится Летний сад?,Where is the Summer garden?
3,Какой город является столицей Туркмении?,Which city is the capital of Turkmenistan?
4,В каком городе издавалась с 1857 г. А. Герцено...,In which city was the first Russian revolution...


In [ ]:
X_train = [sentence.split() for sentence in data_train['question_text']]
y_train = [sentence.split() for sentence in data_train['question_eng']]

X_test = [sentence.split() for sentence in data_test['question_text']]
y_test = [sentence.split() for sentence in data_test['question_eng']]

print(f"{X_train[0], y_train[0]=} \n")
print(f"{len(X_train)=} {len(y_train)=}")
print(f"{len(X_test)=} {len(y_test)=}")

X_train[0], y_train[0]=(['Что', 'может', 'вызвать', 'цунами?'], ['What', 'can', 'cause', 'a', 'tsunami?']) 

len(X_train)=2330 len(y_train)=2330
len(X_test)=580 len(y_test)=580


In [ ]:
ru_vocab = build_vocab_from_iterator(X_train, specials=['<UNK>', '<PAD>', '<SOS>', '<EOS>'])
ru_vocab.set_default_index(0)

en_vocab = build_vocab_from_iterator(y_train, specials=['<UNK>', '<PAD>', '<SOS>', '<EOS>'])
en_vocab.set_default_index(0)

print(f"{len(ru_vocab)=}")
print(f"{len(en_vocab)=} \n")
print(f"Максимальная длина RU: {max(len(sentence) for sentence in X_train)}")
print(f"Максимальная длина EN: {max(len(sentence) for sentence in y_train)}")

len(ru_vocab)=6548
len(en_vocab)=5189 

Максимальная длина RU: 25
Максимальная длина EN: 31


In [ ]:
class RuEnDataset(Dataset):
    def __init__(self, X, y, X_transform, y_transform):
        self.X = X
        self.y = y
        self.X_transform = X_transform
        self.y_transform = y_transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (self.X_transform(self.X[index]), self.y_transform(self.y[index]))

In [ ]:
X_transform = T.Sequential(
    T.VocabTransform(ru_vocab),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor())

y_transform = T.Sequential(
    T.VocabTransform(en_vocab),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor())

In [ ]:
dataset_train = RuEnDataset(X_train, y_train, X_transform, y_transform)
dataset_train[0]

(tensor([   2,   31,  675, 4057, 6433,    3]),
 tensor([   2,    7,  145,  240,   25, 5064,    3]))

In [ ]:
dataset_test= RuEnDataset(X_test, y_test, X_transform, y_transform)
dataset_test[0]

(tensor([  2,   8,  22,  46, 135, 112,   0,   3]),
 tensor([  2,  13,  24,  20,   4,  42,   0, 116,  51,  47,   3]))

In [ ]:
class Collator:
    def __init__(self, pad_idx, batch_first=True):
        self.pad_idx = pad_idx
        self.batch_first = batch_first
    def __call__(self, batch):
        x = [x[0] for x in batch]
        y = [x[1] for x in batch]
        x = pad_sequence(x, batch_first = self.batch_first, padding_value=self.pad_idx)
        y = pad_sequence(y, batch_first = self.batch_first, padding_value=self.pad_idx)
        return (x,y)

In [ ]:
train_loader = DataLoader(dataset_train, batch_size=256, collate_fn=Collator(batch_first=True, pad_idx=1), shuffle=True)

In [ ]:
next(iter(train_loader))[0].shape

torch.Size([256, 25])

In [ ]:
next(iter(train_loader))[0].shape

torch.Size([256, 19])

In [ ]:
next(iter(train_loader))[0].shape

torch.Size([256, 27])

<p class="task" id="2"></p>

### 2
Создайте и обучите модель машинного перевода, используя архитектуру Encoder-Decoder на основе RNN с использованием механизма аддитивного внимания. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

### 3
Сгенерируйте перевод при помощи обученной модели и визуализируйте матрицу внимания, в которой отображено, на какие слова из исходного предложения модель обращала внимание при генерации очередного слова в переводе.

- [ ] Проверено на семинаре

## Обратная связь
- [ ] Хочу получить обратную связь по решению